## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [5]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl, fs = 125)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl, fs = 125):
    """
    Makes pulse rate estimation and confidence estimation

    Args:
        data_fl: (str) filepath to a troika .mat file.
    Returns:
        errors: Calculated errors for each heart rate
        confidence: Calculated confidence for each heart rate
    """
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # Compute pulse rate estimates and estimation confidence.
    # Bandpass filter
    ppg, accx, accy, accz = bandpass_filter(ppg, fs), bandpass_filter(accx, fs), bandpass_filter(accy, fs), bandpass_filter(accz, fs)
    
    # Magnitude of acceleration as 3D
    acc = np.sqrt(accx**2 + accy**2 + accz**2)
    
    # Load ground truth as column vector and convert to row vector
    ground_truth = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1) 
    
    # Calculate heart rate prediction, confidence and errors
    predictions, confidence = predict_heart_rate(acc, ppg, fs)
    errors = np.abs(np.subtract(predictions, ground_truth))
    
    # Return per-estimate mean absolute error and confidence
    errors, confidence = np.array(errors), np.array(confidence)
    return errors, confidence

def bandpass_filter(signal, fs, pass_band=(40/60.0, 240/60.0)):
    """
    Filter bandpass for unrealistic heart rate values.
    Args:
        signal: numpy array. For ppm or acc signal
        fs: int. Sampling frequency in Hz
        pass_band: tuple. band width for filter
    Returns:
        predictions: heart rate predicitons 
        confidence: confidence estimates
    """
    b, a = scipy.signal.butter(3, pass_band, btype='bandpass', fs=fs) # 3: Order of the filter 
    return scipy.signal.filtfilt(b, a, signal)

def predict_heart_rate(acc, ppg, fs, window_length = 8, window_shift = 2, min_bandpass = 40/60.0, max_bandpass = 240/60.0):
    """
    Predict heart rate and calculate confidence using ppg and accelerometer data

    Args:
        acc: numpy array. Acceleration signal file
        ppg: numpy array. PPG signal file
        fs: int. Sampling frequency in Hz
        window_length_s: int. Window_length in seconds
        window_shift_s: int. Window shift in seconds

    Returns:
        predictions: list. Predicited heart rates
        confidence: list. Estimated confidences
    """
    # Define window shapes with given values to function
    window_length = window_length * fs
    window_shift = window_shift * fs
    
    # Create empty lists to store calculations
    bpm_predictions = []
    confidence = []
    
    # Make calculations with shifting windows
    for origin in range(0, len(ppg) - window_length + 1, window_shift):
        
        # Estimation for current window
        ppg_window = ppg[origin:origin+window_length]
        acc_window = acc[origin:origin+window_length]
        
        # Fourier transform
        ppg_freqs, ppg_fft = fourier_transform(ppg_window, fs)
        acc_freqs, acc_fft = fourier_transform(acc_window, fs)
        
        # Remove unwanted frequencies 
        ppg_fft[ppg_freqs <= min_bandpass] = 0.0
        ppg_fft[ppg_freqs >= max_bandpass] = 0.0
        acc_fft[acc_freqs <= min_bandpass] = 0.0
        acc_fft[acc_freqs >= max_bandpass] = 0.0
    
        # Pick frequency with largerst FFT coefficient
        ppg_freq = ppg_freqs[np.argmax(ppg_fft, axis=0)]  
        acc_freq = acc_freqs[np.argmax(acc_fft, axis=0)]
        
        freq = ppg_freq
        conf = confidence_calculator(ppg_freqs, ppg_fft, ppg_freq)
        
        # If the dominant accelerometer frequency is the same as the PPG, estimated heart rate is biased.
        # Make new estimation sor second suitable frequency.
        if abs(ppg_freq - acc_freq) == 0:
            
            # Calculate frequency with second largerst FFT
            sec_freq = ppg_freqs[np.argsort(ppg_fft, axis=0)[-2]]
            sec_conf = confidence_calculator(ppg_freqs, ppg_fft, sec_freq)
            
            # If new freq is larger, choose this as freq
            if sec_conf > conf:
                freq, conf = sec_freq, sec_conf
        
        bpm_predictions.append(freq * 60) # Convert to bpm (from Hz)
        confidence.append(conf)
        
    return bpm_predictions, confidence

def fourier_transform(signal, fs):
    """
    Calculate the Fast Fourier Transfrom of given sequence
    Args:
        signal: numpy array. For ppm or acc signal
        fs: int. Sampling frequency in Hz
    Returns:
        freqs: numpy array. Frequency bins 
        fft: float. Magnitude of FFT
    """ 
    n_samples = len(signal)  # No zero padding
    freqs = np.fft.rfftfreq(n_samples, 1/fs)
    fft = np.abs(np.fft.rfft(signal, n_samples))
    return freqs, fft

def confidence_calculator(freqs, fft, freq, window_half_width=1):
    """
    Calculate confidence by Fourier transformes data inputs.
    Args:
        freqs: numpy array. Frequency bins 
        fft: float. Magnitude of FFT
        freq: float. Frequence for calculation confidence.
    Returns:
        confidence: float.
    """ 
    window = (freqs > freq - window_half_width) & (freqs < freq + window_half_width)  
    confidence = np.sum(fft[window]) / np.sum(fft)
    return confidence

In [6]:
Evaluate()

15.726958002724126

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

In [19]:
!pip list

Package                       Version    
----------------------------- -----------
altair                        1.2.1      
asn1crypto                    0.22.0     
atari-py                      0.1.7      
atomicwrites                  1.3.0      
attrs                         19.1.0     
audioread                     2.1.6      
av                            0.3.3      
awscli                        1.16.17    
backcall                      0.1.0      
backports.functools-lru-cache 1.4        
backports.weakref             1.0rc1     
beautifulsoup4                4.6.0      
bleach                        1.5.0      
blinker                       1.4        
bokeh                         0.12.13    
boto                          2.48.0     
boto3                         1.9.7      
botocore                      1.12.7     
Box2D                         2.3.2      
Box2D-kengz                   2.3.3      
bresenham                     0.2        
bz2file                       0.98

# Code Description

This project performs pulse rate estimation using the Troika dataset.
The code estimates heart rate without motion artefacts. 
You can run pulse rate algorithm by simply use **Evaluate()** funciton. It returns an aggregate error metric.

Required imports: 
- glob: 2.0.0, 
- numpy: 1.12.1, 
- scipy: 1.2.1, scipy.io, scipy.signal

You can change this values before running: 
- Bandpass filter range: predict_heart_rate(), bandpass_filter() functions. We used recommended values for this project in project readme file.
- window_length, window_shift: predict_heart_rate() function. Wider window length should improve estimation performance, because narrow window should miss some peaks near the border.

# Data Description

Used dataset in this project called as TROIKA. Zhang and co-authors presented this dataset in the paper TROIKA: A general framework for heart rate monitoring using wrist-type photoplethysmographic signals during intensive physical exercise.

Dataset contains 12 persons' ppg and three dimensions(x, y, z) accelerometer signal values.
All subjects are male and they are between 18-35 years old.
The data collected by wearing a wrist device while resting(30 sec) walking, running(4 min) and during a brief cooldown(30 sec) on a treadmill. All signals were sampled at 125 Hz.

The dataset also includes reference heart rate values calculated from the ECG signal for using as ground-truth.

**Short comings:** All of the data collected from only male subjects and age distribution is too restricted. This may not does not reflect the population. Also we don't know about skin color of the subjects. Dark and light skin may differ.
For better estimation, dataset must contain diversity.

# Algorithhm Description

**Phisiology** of behind this algorithm is movement of the blood in the vessels. During systole, capillaries fill with blood and during diastole, blood inside the capillaries decreases. The PPG sensor uses a green light. This light absorbed by the RBC. The changes in the amount of reflected light is detected by the sensor. By this data, we understand the phase of the heart cycle.

**The algorithm** is composed of several functions. Calculates heart rate without artefacts.
We load ppg and acc signal values from files, bandpass filtering on raw signals between 40 and 240 BPM and apply the Fourier transform. This cut-off values determined by physiologic heart rates. The algorithm computes largest FFT coefficient, pick the PPG frequency with this as the best estimate. 
After finding predicted frequency, for understanding that this value not comes from movement artefacts, compare this value with acc signals frequency. If difference equals 0, the algorithm tests the PPG frequency with the second largest FFT coefficient.

**The algorithm outputs** the array of mean absolute errors calculated between predicted vs ground-truth heart rate array and array of confidence score. The confidence includes how much energy in the frequency spectrum is concentrated near the pulse rate estimate. The confidence represents the confidence that the estimated pulse rate represents a correct estimated pulse rate.

**Insightful caveats and failure-modes of the algorithm:** The algorithm is based on estimation of strong frequency spectral in the ppg signals relative to the entire frequency spectral of the ppg signals. The wearer's motion can produce strong signals which can be confused for the pulse rate resulting in algorithm failure.

# Algorithm Performance

We calculate the Mean Absolute Error between the estimates and the ground-truth labels. 

Training: **16.141985488089304**
Test: 10.04*

The algorithm performed relatively well on test data. 
Having a lower error during testing might mean that the test examples are less challenging than the one used for training. This can also means that there is data leakage. For avoid leakage, make sure that using leave-one-subject-out cross-validation for testing.

This dataset generated from only 12 male patients. Women data distribution may differs with men. And age distribution is not comprehensive. Also, activities only restricted with running, not include swimming, jogging, biking etc. This lacking data types may decrease performance of algorithm in different subjects that mentioned above.

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 